In [1]:
import os
import sys
sys.path.append('/Automatic-Circuit-Discovery/')
sys.path.append('..')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor
import numpy as np
import matplotlib.pyplot as plt
import json

from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.docstring.utils import get_all_docstring_things
from utils.prune_utils import get_3_caches, split_layers_and_heads
from ACDCPPExperiment import ACDCPPExperiment
device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(device)

/var/folders/tn/nyh0947d6sngjfvtp7hbb0jr0000gn/T/ipykernel_38888/1557166804.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/var/folders/tn/nyh0947d6sngjfvtp7hbb0jr0000gn/T/ipykernel_38888/1557166804.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')
/Users/canrager/.pyenv/versions/acdc_env/envs/acdcpp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [2]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

Loaded pretrained model attn-only-4l into HookedTransformer
Moving model to device:  cpu


In [3]:
def abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

# ACDC++ run

In [5]:
RUN_NAME = "docstring_absval_edge"
acdcpp_docstring_convergence_thresh = 0.03
acdc_paper_thresh = 0.067
num_thresholds = 1

ACDCPP_THRESHOLDS = np.linspace(
    0.75 * acdcpp_docstring_convergence_thresh,
    1.25 * acdcpp_docstring_convergence_thresh,
    num_thresholds
)
ACDC_THRESHOLDS = np.linspace(
    0.75 * acdc_paper_thresh,
    1.25 * acdc_paper_thresh,
    num_thresholds
)

print(f"{ACDCPP_THRESHOLDS=}")
print(f"{ACDC_THRESHOLDS=}")

ACDCPP_THRESHOLDS=array([0.0225])
ACDC_THRESHOLDS=array([0.05025])


In [6]:
combined_exp = ACDCPPExperiment(
    tl_model,
    test_data,
    test_patch_data,
    acdc_metric=test_metrics['docstring_metric'],
    acdcpp_metric=abs_docstring_metric,
    acdc_thresholds=ACDC_THRESHOLDS,
    acdcpp_thresholds=ACDCPP_THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)
pruned_heads, num_passes, pruned_attrs = combined_exp.run()

self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:])
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


Edge pruning:  13%|█▎        | 26/193 [00:00<00:00, 256.97it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ACDC++:   0%|          | 0/1 [00:00<?, ?it/s]

acdcpp_threshold=0.0225


acdc_threshold=0.05025
self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:])


ACDC++:   0%|          | 0/1 [11:08<?, ?it/s]


KeyboardInterrupt: 